# Курсовик по предмету "Рекомендательные системы" Гладышев В.В.

### EDA, фильтрация, feature engineering:

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/coursework/cw_eda_glvv.ipynb

### библиотека с функциями метрик:

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/src/metrics.py

### подготовка данных: 

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/src/utils.py

### класс с моделями:

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/src/recommenders.py